## apoptosis model - increasing Bcl2 copy number

In [2]:
using Plots 
using CSV
using Distributions
using Random
using DataFrames
using JLD2
using DelimitedFiles
using FileIO
using Sundials
using LinearAlgebra
using DifferentialEquations

In [3]:
#set up where CSV2Julia is
locationOfCSV2Julia="csv2model-multiscale.py"

#identify the three CSV sheets that describe the model
reactionsFile="reactions.csv"
parametersFile="parameters.csv"
rateLawsFile="rateLaws.csv"

#build a command to execute csv2julia
arguments=[reactionsFile, parametersFile, rateLawsFile, "odeApoptosis.jl"]
cmd=`python3 $locationOfCSV2Julia $arguments`

#lets run csv2julia (requires python to be installed)
run(cmd)

inline
Running CSV2JuliaDiffEq with parameters hard-coded into the CSV file, if this is not correct, re-run with the 5th argument set to 'scan' or 'param'
Opening rateLaws.csv as rate law file
Opening parameters.csv as parameters file
Opening reactions.csv as reactions file


Process(`python3 csv2model-multiscale.py reactions.csv parameters.csv rateLaws.csv odeApoptosis.jl`, ProcessExited(0))

In [4]:
#CV =  coefficient of variation
preConcCV=0.32 # how much should we distribute starting parameters for each cell
# We need a concentration CV of about 32% (if we were doing parameters it would be 11%ish)
# I've tried 32% and it seems too high, the effect of variability ends up way greater than the effect of
# the BCL-2 inhibition so I reduced this to 11%... can we justify that cell-to-cell variability is lower
# between cell lines than between primary cells? Probably.
# as reported here: https://www.pnas.org/content/115/12/E2888

#DISincrease=0.000001

0.32

In [5]:
function myLogNormal(m,std)
    γ = 1+std^2/m^2
    μ = log(m/sqrt(γ))
    σ = sqrt(log(γ))
    return LogNormal(μ,σ)
end

myLogNormal (generic function with 1 method)

In [6]:
#this function takes some initial conditions and distributes them by a set CV, returning new initial conditions
function sampleInit(y0,cv)
    #note mean of distribution is 1 so CV = standard deviation
    distribution=Truncated(myLogNormal(1,cv),0,Inf)
    scaling=rand(distribution, 1)
    scalingMatrix=[ rand(distribution, 1)[1] for i=1:length(y0)]
    ynew=y0.*scalingMatrix
    return ynew
end

sampleInit (generic function with 1 method)

In [7]:
odeFile=include("odeApoptosis.jl")
include("variableNames.jl")
p=1
maxTimeSS=100000.0
maxTimeTC=144*60.0
params=1

1

In [8]:
# fix species
include("fixSpecies.jl")
fixSpecies("odeApoptosis.jl","odeApoptosisFixed.jl",1)
odeFile = include("odeApoptosisFixed.jl")

odeApoptosis! (generic function with 1 method)

In [9]:
## Bcl2t values:

# WT  = 277
# 1.5 = 415.5
# 2.0 = 554
# 2.5 = 692.5
# 3.0 = 831
# 3.5 = 969.5

# replace xxx with value for copy number

function initConditionsApop(y0,syms)
   #units: M
   #source: the supplement of Albeck et al 2008 PLOS Biology
   y0[findfirst(isequal("L"),syms)]=0
   y0[findfirst(isequal("R"),syms)]=1000
   y0[findfirst(isequal("flip"),syms)]=2000
   y0[findfirst(isequal("pC8"),syms)]=10000
   y0[findfirst(isequal("BAR"),syms)]=1000
   y0[findfirst(isequal("pC3"),syms)]=10000
   y0[findfirst(isequal("pC6"),syms)]=10000
   y0[findfirst(isequal("XIAP"),syms)]=100000
   y0[findfirst(isequal("PARP"),syms)]=1000000
   y0[findfirst(isequal("Bid"),syms)]=60000
   y0[findfirst(isequal("Bcl2c"),syms)]=20000
   y0[findfirst(isequal("Bax"),syms)]=80000
   y0[findfirst(isequal("Bcl2"),syms)]=30000
   y0[findfirst(isequal("Mito"),syms)]=500000
   y0[findfirst(isequal("mCytoC"),syms)]=500000
   y0[findfirst(isequal("mSMac"),syms)]=100000
   y0[findfirst(isequal("Apaf"),syms)]=100000
   y0[findfirst(isequal("Procasp9"),syms)]=100000
   #this is new and will be replaced by cRel control in multiscale modeling
   y0[findfirst(isequal("Bcl2t"),syms)]=277
    return y0
end

initConditionsApop (generic function with 1 method)

In [10]:
Bcl2tIndex=findfirst(isequal("Bcl2t"),syms)

59

In [11]:
global f=ODEFunction(odeFile,syms=Symbol.(syms))

(::ODEFunction{true, SciMLBase.FullSpecialize, typeof(odeApoptosis!), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Vector{Symbol}, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}) (generic function with 1 method)

In [12]:
import Base.Threads

In [13]:
conditions=["WT","Bcl2_mut"]
bcl2Vals=[1,1.5]

2-element Vector{Float64}:
 1.0
 1.5

In [14]:
first_cell = 1
last_cell = 1000

1000

### Steady state

In [14]:
function genSSconds(Bcl2,mut)
    y0=zeros(length(syms))
    y0=initConditionsApop(y0,syms)
    y0[findfirst(isequal("L"),syms)]=0
    y0[findfirst(isequal("Bcl2t"),syms)]=277*Bcl2

    #solve the steady state
    prob=ODEProblem(f,y0,(0.0,maxTimeSS),abstol=1e-5,reltol=1e-3, maxiters=1e10)
    solss=solve(prob, Rosenbrock23(), saveat=maxTimeSS/1000)

    #dynamic phase, use SS solution as initial conditions
    y0_ss=vec(convert(Array, solss[:,end]))    
    y0_ss[findfirst(isequal("L"),syms)]=1

    # save y0 as csv
    writedlm("steady_state_"*mut*".csv", y0_ss, ',')
    println("steady state conditions saved")
end

genSSconds (generic function with 1 method)

In [ ]:
genSSconds(1,"WT")

In [ ]:
genSSconds(1.5,"Bcl2")

In [ ]:
function genRunConds(Bcl2, condition_folder)
    Random.seed!(1234) #for reproducibility
    for i in first_cell:last_cell
        #import SS conds
#         y0_ss = readdlm("steady_state_"*condition_folder*".csv")
        y0_ss = readdlm("steady_state_WT.csv")
        #set L value
        y0_ss[findfirst(isequal("L"),syms)]=1
        
        #distribute all initial conditions
        y0_ss = sampleInit(y0_ss,preConcCV)
#         thisDist=Truncated(myLogNormal(1,preConcCV),0,Inf)
#         for j in 1:length(y0_ss)
#             x = rand(thisDist,1)
#             y0_ss[j] = y0_ss[j]*x[1]
#         end
                
        # set Bcl2 value
        y0_ss[findfirst(isequal("Bcl2t"),syms)]=277*Bcl2

        # save y0 as csv
        writedlm("SS_conds/"*condition_folder*"/SS_conds_"*string(i)*".csv", y0_ss, ',')    
        println("Run conditions saved")
    end
end

In [ ]:
function runSimSS(first_cell, last_cell, Bcl2, thisCondition)
    Threads.@threads for i in first_cell:last_cell
        solveSS(i, Bcl2, thisCondition)
    end
end

In [ ]:
y0_WT = vec(readdlm("steady_state_WT.csv"))

### Run full sim

In [15]:
function genInitConds(first_cell, last_cell, Bcl2, thisCondition)
    #import SS conds
    y0_ss = vec(readdlm("steady_state_"*thisCondition*".csv"))
    
    for i in first_cell:last_cell
        #distribute all initial conditions
        y0_init = sampleInit(y0_ss,preConcCV)
                
        # set Bcl2 value
        y0_init[findfirst(isequal("Bcl2t"),syms)]=277*Bcl2

        # save y0 as csv
        writedlm("SS_conds/"*thisCondition*"/SS_conds_"*string(i)*".csv", y0_init, ',')
    end
    println("conditions for "*thisCondition*" saved")
end

genInitConds (generic function with 1 method)

In [16]:
for condIndex in 1:length(conditions)
    #set seed
    Random.seed!(1234) #for reproducibility
    #get conditions
    thisCondition=conditions[condIndex]
    thisBCL2Scale=bcl2Vals[condIndex]
    #create folder to save SS conditions
    mkpath("SS_conds/"*thisCondition*"/")
    #run simulations
    genInitConds(first_cell, last_cell, thisBCL2Scale, thisCondition)
end

conditions for WT saved
conditions for Bcl2_mut saved


In [17]:
#find cleaved parp
speciesToPlot="CPARP"
#find total PARP
denominator=["PARP","PARP_C3","CPARP"]
CPARPIndex=findfirst(isequal(speciesToPlot),syms)
denominatorIndexes=Array{Int64}(undef,length(denominator))
for i in 1:length(denominator)
    thisIndex=findfirst(isequal(denominator[i]),syms)
    denominatorIndexes[i]=convert(Int64,thisIndex)
end

In [22]:
function calcT2D(sol, i)
    cParray=zeros(1,Int(maxTimeTC)+1)
    thisNumerator=sol[CPARPIndex,:] #cPARP
    thisDenominatorTCs=sol[denominatorIndexes,:]
    thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
    parpPercentage=thisNumerator./thisDenominator'
    cParray=[cParray;parpPercentage']
    if cParray[end] > 0.01
        T2Death = findfirst(x->x>0.01, cParray)            
        T2D_t = T2Death[2]
        T2D_df = DataFrame(Cell="cell_"*string(i), T2D=T2D_t)
        CSV.write("T2D_res/T2D_res_"*thisCondition*"/cell_"*string(i)*"_res.csv", T2D_df)
    else
    end
end

calcT2D (generic function with 2 methods)

In [24]:
function solveCell(i, Bcl2, thisCondition)    
        
    for i in first_cell:last_cell
        
        y0_init = vec(readdlm("SS_conds/"*thisCondition*"/SS_conds_"*string(i)*".csv"))
        
        prob=ODEProblem(f,y0_init,(0.0,maxTimeTC),abstol=1e-5,reltol=1e-3, maxiters=1e10)
        sol=solve(prob, Rosenbrock23(), saveat=1.0, progress = true)

        calcT2D(sol, i)

    end
    println(thisCondition*" finished")
end

solveCell (generic function with 1 method)

In [20]:
function runSim(first_cell, last_cell, Bcl2, thisCondition)
    Threads.@threads for i in first_cell:last_cell
#         try 
            solveCell(i, Bcl2, thisCondition)
#         catch err
#             println("cell "*string(i)*"failed, continuing.")
#             #println(err)
#             continue
#         end
    end
end

runSim (generic function with 1 method)

### Run simulation

In [ ]:
for condIndex in 1:length(conditions)
    #set seed
    Random.seed!(1234) #for reproducibility
    #get conditions
    thisCondition=conditions[condIndex]
    thisBCL2Scale=bcl2Vals[condIndex]
    #create folder to save SS conditions
    mkpath("T2D_res/T2D_res_"*thisCondition*"/")
    #run simulations
    runSim(first_cell, last_cell, thisBCL2Scale, thisCondition)
end

In [ ]:
#set seed
Random.seed!(1234) #for reproducibility
#get conditions
thisCondition=conditions[2]
thisBCL2Scale=bcl2Vals[2]
#create folder to save SS conditions
#mkpath("T2D_res/T2D_res_"*thisCondition*"/")
#run simulations
runSim(first_cell, last_cell, thisBCL2Scale, thisCondition)

### New Plotting

In [ ]:
colorArray=palette(:seaborn_colorblind)
for condIndex in 1:length(conditions)
    thisCondition=conditions[condIndex]
    thisBCL2Scale=bcl2Vals[condIndex]
    #plotly()
    CPARPIndex=findfirst(isequal("CPARP"),syms)

    #plotVar=plot(legend=false,foreground_color_subplot=:white,title="all cells cparp percentage")
    plotT2D=Plots.plot!(title="all cells cparp percentage")
    for i in first_cell:last_cell
        if isfile("solutions/"*thisCondition*"/cell_"*string(i)*".jld2") == true
            sol = load("solutions/"*thisCondition*"/cell_"*string(i)*".jld2", "solution")

            #calculate cleaved parp as a percentage of total and store in the array.
            thisNumerator=sol[CPARPIndex,:] #cPARP
            thisDenominatorTCs=sol[denominatorIndexes,:]
            thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
            parpPercentage=thisNumerator./thisDenominator'

                plot!(plotT2D,parpPercentage.*100,color=colorArray[condIndex],legend=false,
            xticks=(0:1000:8000))
        else
            continue
        end
    end
end
display(plotT2D)

### Code for importing results and plotting

In [ ]:
function T2D(mut_folder, first_cell, last_cell)
    T2D_array = zeros(first_cell, last_cell)
    for i in first_cell:last_cell
       if isfile("solutions/"*mut_folder*"/cell_"*string(i)*".jld2") == true
        cParray=zeros(1,Int(maxTimeTC)+1)
        sol = load("solutions/"*mut_folder*"/cell_"*string(i)*".jld2", "solution")    
            thisNumerator=sol[CPARPIndex,:] #cPARP
            thisDenominatorTCs=sol[denominatorIndexes,:]
            thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
            parpPercentage=thisNumerator./thisDenominator'
            cParray=[cParray;parpPercentage']
            if cParray[end] > 0.01
                T2Death = findfirst(x->x>0.01, cParray)            
                T2D_array[i] = T2Death[2]
            else
                continue
            end
        else
            continue
        end
    end
    T2D_array_df = DataFrame(T2D_array', :auto)
    return(T2D_array_df)
end

In [ ]:
T2D_res_WT = T2D("WT",1,100) 

In [ ]:
T2D_res_Bcl2mut = T2D("BCL2_mut",1, 1000) 

In [ ]:
histogram(T2D_res_WT[!, :x1], 
            bins=200, size=(900,700), 
            label="WT", 
            xlims = (55*60,65*60), 
            title="Time to cell death",
            xlab = "time hours") 
s=string.(collect(0:5:100))
plot!(xticks = (0:(60*5):(60*100),s),xtickfontsize=14,ytickfontsize=14,xguidefontsize=18,yguidefontsize=18)

In [ ]:
mut_folder="WT"

In [ ]:
#plotly()
CPARPIndex=findfirst(isequal("CPARP"),syms)

#plotVar=plot(legend=false,foreground_color_subplot=:white,title="all cells cparp percentage")
plotT2D=Plots.plot(title="all cells cparp percentage WT")
for i in first_cell:last_cell
    if isfile("solutions/"*mut_folder*"/cell_"*string(i)*".jld2") == true
        sol = load("solutions/"*mut_folder*"/cell_"*string(i)*".jld2", "solution")

        #calculate cleaved parp as a percentage of total and store in the array.
        thisNumerator=sol[CPARPIndex,:] #cPARP
        thisDenominatorTCs=sol[denominatorIndexes,:]
        thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
        parpPercentage=thisNumerator./thisDenominator'
        plot!(plotT2D,parpPercentage.*100,label="cell: "*string(i),legend=false,
        xticks=(0:1000:8000))
    else
        continue
    end
end
display(plotT2D)

In [ ]:
#plotly()
CPARPIndex=findfirst(isequal("CPARP"),syms)

#plotVar=plot(legend=false,foreground_color_subplot=:white,title="all cells cparp percentage")
plotT2D=Plots.plot(title="all cells cparp percentage WT")
for i in first_cell:last_cell
    if isfile("solutions/"*mut_folder*"/cell_"*string(i)*".jld2") == true
        sol = load("solutions/"*mut_folder*"/cell_"*string(i)*".jld2", "solution")

        #calculate cleaved parp as a percentage of total and store in the array.
        thisNumerator=sol[CPARPIndex,:] #cPARP
        thisDenominatorTCs=sol[denominatorIndexes,:]
        thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
        parpPercentage=thisNumerator./thisDenominator'
        plot!(plotT2D,parpPercentage.*100,label="cell: "*string(i),legend=false,
        xticks=(0:1000:8000))
    else
        continue
    end
end
display(plotT2D)

In [ ]:
function calcT2D(mut_folder, cell_num)
    cParray=zeros(1,Int(maxTimeTC)+1)
    sol = load("solutions/"*mut_folder*"/cell_"*string(cell_num)*".jld2", "solution")
    thisNumerator=sol[varIndex,:] #cPARP
    thisDenominatorTCs=sol[denominatorIndexes,:]
    thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
    parpPercentage=thisNumerator./thisDenominator'
    cParray=[cParray;parpPercentage']
    T2D_death = findfirst(x->x>0.01, cParray)
    #T2D_t = T2D_death[1]
    return T2D_death
end

In [ ]:
cParray=zeros(1,Int(maxTimeTC)+1)
thisNumerator=sol[varIndex,:] #cPARP
thisDenominatorTCs=sol[denominatorIndexes,:]
thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
parpPercentage=thisNumerator./thisDenominator'
cParray=[cParray;parpPercentage']
T2D_death = findfirst(x->x>0.01, cParray)

In [ ]:
#plotly()
CPARPIndex=findfirst(isequal("CPARP"),syms)

plotVar_mut=Plots.plot(title="all cells cparp percentage Bcl2t x1.5")
for i in first_cell:last_cell
    sol = load("solutions/"*mut_folder*"/cell_"*string(i)*".jld2", "solution")

    #calculate cleaved parp as a percentage of total and store in the array.
    thisNumerator=sol[CPARPIndex,:] #cPARP
    thisDenominatorTCs=sol[denominatorIndexes,:]
    thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
    parpPercentage=thisNumerator./thisDenominator'
    plot!(plotVar_mut,parpPercentage.*100,label="cell: "*string(i), legend=false,
    xticks=(0:1000:8000))
end

In [ ]:
function res_mod(res,mut)
    rowNumberList=rownumber.(eachrow(res))
    res.Cell_id="cell_".*string.(rowNumberList)
    rename!(res, :x1 => mut)
    sort!(res, [mut], rev=true)
    return(res)
end

In [ ]:
WT_res_mod = res_mod(T2D_res_WT, "WT")

In [ ]:
Bcl2mut_res_mod = res_mod(T2D_res_bcl2mut, "Bcl2_mut")

In [ ]:
CSV.write("WT_res_mod.csv", WT_res_mod)
CSV.write("Bcl2_res_mod.csv", Bcl2mut_res_mod)

In [ ]:
WT_res_mod = DataFrame(CSV.File("WT_res_mod.csv"))

In [ ]:
Bcl2mut_res_mod = DataFrame(CSV.File("Bcl2_res_mod.csv"))

In [ ]:
WT_Bcl2_res = innerjoin(WT_res_mod, Bcl2mut_res_mod, on = :Cell_id)

In [ ]:
plotb1=plot(ylim = (0,1000))
        for row in eachrow(WT_res)
            plot!(plotb1,[0,row["T2D"]],color=:green,legend = false,lw=0.5)
        end   

In [ ]:
bar(WT_Bcl2_res.Bcl2_mut, orientation=:h, size=(750,750), linewidth = 0, color=:green, alpha=0.5, xlim=(1500,6000), label="Bcl2 x1.5")
bar!(WT_Bcl2_res.WT, orientation=:h, size=(750,750), linewidth = 0, color=:black, alpha=0.5, xlim=(1500,6000), label="WT")

In [ ]:
TTD_array=[]

In [ ]:
first_cell=1
last_cell=100

In [ ]:
#now lets get the time to death
#filesInDir=readdir("outputFiles/apoptosis_ave_Bcl2/")
for i in first_cell:last_cell
    sol = load("solutions/WT/cell_"*string(i)*".jld2", "solution")
    cPARPArray=zeros(1,Int(maxTimeTC)+1)
    #calculate cleaved parp as a percentage of total and store in the array.
    thisNumerator=sol[varIndex,:] #cPARP
    thisDenominatorTCs=sol[denominatorIndexes,:]
    thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
    parpPercentage=thisNumerator./thisDenominator'
    cPARPArray=[cPARPArray;parpPercentage']
    time_point = findfirst(x->x>0.1, cPARPArray)
    TTD = sol.t[time_point[2]]
    append!(TTD_array, TTD)
end